In [1]:
import numpy as np
import pandas as pd
import matplotlib
import cv2
import torchvision
import torch
from torch.autograd import Variable
from torch import Tensor, cat, sigmoid, exp, stack, max, meshgrid, linspace, arange
from model import Darknet, Tail, Head


from weight_formater import darknet, tail, weight, weights

odict_keys(['intro.0.weight', 'intro.1.weight', 'intro.1.bias', 'intro.1.running_mean', 'intro.1.running_var', 'intro.1.num_batches_tracked', 'module_list.0.0.0.weight', 'module_list.0.0.1.weight', 'module_list.0.0.1.bias', 'module_list.0.0.1.running_mean', 'module_list.0.0.1.running_var', 'module_list.0.0.1.num_batches_tracked', 'module_list.0.1.0.weight', 'module_list.0.1.1.weight', 'module_list.0.1.1.bias', 'module_list.0.1.1.running_mean', 'module_list.0.1.1.running_var', 'module_list.0.1.1.num_batches_tracked', 'module_list.0.1.3.weight', 'module_list.0.1.4.weight', 'module_list.0.1.4.bias', 'module_list.0.1.4.running_mean', 'module_list.0.1.4.running_var', 'module_list.0.1.4.num_batches_tracked', 'module_list.1.0.0.weight', 'module_list.1.0.1.weight', 'module_list.1.0.1.bias', 'module_list.1.0.1.running_mean', 'module_list.1.0.1.running_var', 'module_list.1.0.1.num_batches_tracked', 'module_list.1.1.0.weight', 'module_list.1.1.1.weight', 'module_list.1.1.1.bias', 'module_list.1.1

In [ ]:
number_of_classes = 1

In [2]:
len(weight)

5058084

In [3]:
len(weights)

61576342

In [ ]:
image = cv2.imread("/home/ivan/Desktop/BN-UQ280_FEDUCI_GR_20170810142213.jpg")
image = cv2.resize(image, (416, 416), interpolation=cv2.INTER_LINEAR)
image_tensor = image/255.0     
image_tensor = Variable(torch.from_numpy(image_tensor).float())
image_tensor = image_tensor.permute(2, 0, 1)

In [ ]:
head = Head(Tensor([[120, 93],  [84,191], [238,186]]),1)
anchors=Tensor([[120, 93],  [84,191], [238,186]]).view(3, 1, 1, 2)
head_1 = Head(Tensor([[22, 59],  [61, 43],  [50, 97]]),1)
anchors_1=Tensor([[22, 59],  [61, 43],  [50, 97]]).view(3, 1, 1, 2)
head_2 = Head(Tensor([[12, 25],  [23, 16],  [36, 26]]),1)
anchors_2 = Tensor([[12, 25],  [23, 16],  [36, 26]]).view(3, 1, 1, 2)

In [ ]:
dark = tail(darknet(image_tensor.unsqueeze(dim=0)))

In [ ]:
features = dark[0]
features_1 = dark[1]
features_2 = dark[2]

In [ ]:
grid_size = list(features.size()[-2:])
cells_offsets = stack(meshgrid(((arange(0, grid_size[0]))/ 13.),
                               ((arange(0, grid_size[0]))/ 13.)), -1)

grid_size_1 = list(features_1.size()[-2:])
cells_offsets_1 = stack(meshgrid(((arange(0, grid_size_1[0]))/ 26.),
                               ((arange(0, grid_size_1[0]))/ 26.)), -1)

grid_size_2 = list(features_2.size()[-2:])
cells_offsets_2 = stack(meshgrid(((arange(0, grid_size_2[0]))/ 52.),
                               ((arange(0, grid_size_2[0]))/ 52.)), -1)


In [ ]:
features = features.view([-1, len(anchors),number_of_classes + 5] + grid_size) \
        .permute(0, 1, 3, 4, 2) \
        .contiguous()
features_1 = features_1.view([-1, len(anchors), number_of_classes + 5] + grid_size_1) \
        .permute(0, 1, 3, 4, 2) \
        .contiguous()
features_2 = features_2.view([-1, len(anchors), number_of_classes + 5] + grid_size_2) \
        .permute(0, 1, 3, 4, 2) \
        .contiguous()

In [ ]:
centers = sigmoid(features[..., :2]) / Tensor(grid_size) + cells_offsets
sizes = exp(features[..., 2:4]) * anchors
probabilities = sigmoid(features[..., 4:])

centers_1 = sigmoid(features_1[..., :2]) / Tensor(grid_size_1) + cells_offsets_1
sizes_1 = exp(features_1[..., 2:4]) * anchors_1
probabilities_1 = sigmoid(features_1[..., 4:])

centers_2 = sigmoid(features_2[..., :2]) / Tensor(grid_size_2) + cells_offsets_2
sizes_2 = exp(features_2[..., 2:4]) * anchors_2
probabilities_2 = sigmoid(features_2[..., 4:])

In [ ]:
boxes = pd.DataFrame(centers.view(-1,2).detach().numpy() * 416, columns=["center_x","center_y"])
boxes["width"]=0
boxes['height']=0
boxes["obj"] = probabilities.view(-1,2)[...,0].detach().numpy()
boxes[["width","height"]] = sizes.view(-1,2).detach().numpy()
boxes["obj"].max()

In [ ]:
boxes_1 = pd.DataFrame(centers_1.view(-1,2).detach().numpy() * 416, columns=["center_x","center_y"])
boxes_1["width"]=0
boxes_1['height']=0
boxes_1["obj"] = probabilities_1.view(-1,2)[...,0].detach().numpy()
boxes_1[["width","height"]] = sizes_1.view(-1,2).detach().numpy()
boxes_1["obj"].max()

In [ ]:
boxes_2 = pd.DataFrame(centers_2.view(-1,2).detach().numpy() * 416, columns=["center_x","center_y"])
boxes_2["width"]=0
boxes_2['height']=0
boxes_2["obj"] = probabilities_2.view(-1,2)[...,0].detach().numpy()
boxes_2[["width","height"]] = sizes_2.view(-1,2).detach().numpy()
boxes_2["obj"].max()

In [ ]:
boxes =boxes.append(boxes_1).append(boxes_2)

In [ ]:
from src.boxes_manipulations import convert_to_matplotlib
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(1, 1, 1)
for x,y, w, h in boxes.loc[boxes["obj"]>0.1,["center_x", "center_y","width", "height"]].values:
       ax.add_patch(plt.Rectangle((int(x+w/2), int(y+h/2)), int(w), int(h), linewidth=4, fill=False))
plt.imshow(image)
plt.show()

In [ ]:
darknet

In [ ]:
def load_weights(self, weightfile):
        
        #Open the weights file
        fp = open(weightfile, "rb")

        #The first 4 values are header information 
        # 1. Major version number
        # 2. Minor Version Number
        # 3. Subversion number 
        # 4. IMages seen 
        header = np.fromfile(fp, dtype = np.int32, count = 5)
        self.header = torch.from_numpy(header)
        self.seen = self.header[3]
        
        #The rest of the values are the weights
        # Let's load them up
        weights = np.fromfile(fp, dtype = np.float32)
        
        ptr = 0
        for i in range(len(self.module_list)):
            module_type = self.blocks[i + 1]["type"]
            
            if module_type == "convolutional":
                model = self.module_list[i]
                try:
                    batch_normalize = int(self.blocks[i+1]["batch_normalize"])
                except:
                    batch_normalize = 0
                
                conv = model[0]
                
                if (batch_normalize):
                    bn = model[1]
                    
                    #Get the number of weights of Batch Norm Layer
                    num_bn_biases = bn.bias.numel()
                    
                    #Load the weights
                    bn_biases = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
                    ptr += num_bn_biases
                    
                    bn_weights = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
                    
                    bn_running_mean = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
                    
                    bn_running_var = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
                    
                    #Cast the loaded weights into dims of model weights. 
                    bn_biases = bn_biases.view_as(bn.bias.data)
                    bn_weights = bn_weights.view_as(bn.weight.data)
                    bn_running_mean = bn_running_mean.view_as(bn.running_mean)
                    bn_running_var = bn_running_var.view_as(bn.running_var)

                    #Copy the data to model
                    bn.bias.data.copy_(bn_biases)
                    bn.weight.data.copy_(bn_weights)
                    bn.running_mean.copy_(bn_running_mean)
                    bn.running_var.copy_(bn_running_var)
                
                else:
                    #Number of biases
                    num_biases = conv.bias.numel()
                
                    #Load the weights
                    conv_biases = torch.from_numpy(weights[ptr: ptr + num_biases])
                    ptr = ptr + num_biases
                    
                    #reshape the loaded weights according to the dims of the model weights
                    conv_biases = conv_biases.view_as(conv.bias.data)
                    
                    #Finally copy the data
                    conv.bias.data.copy_(conv_biases)
                    
                    
                #Let us load the weights for the Convolutional layers
                num_weights = conv.weight.numel()
                
                #Do the same as above for weights
                conv_weights = torch.from_numpy(weights[ptr:ptr+num_weights])
                ptr = ptr + num_weights

                conv_weights = conv_weights.view_as(conv.weight.data)
                conv.weight.data.copy_(conv_weights)